In [ ]:
import pandas as pd

def read_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print("An error occurred:", e)
        return None
    

In [ ]:
data= read_csv_to_dataframe("/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/102_Data.csv")
path_smote="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote"
path_strat="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/strat"
path_smote_strat="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote_strat"

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

def tune_decision_tree_hyperparameters(X_train, y_train, X_test, y_test):
    param_dist = {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': range(1, 101),
        'min_samples_split': range(2, 101),
        'min_samples_leaf': range(1, 11),
        'max_features': ['sqrt', 'log2'],
        'class_weight': ['balanced', None],
    }

    dt_classifier = DecisionTreeClassifier(random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)


    cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    randomized_search = RandomizedSearchCV(
        dt_classifier, param_distributions=param_dist, cv=cv_strategy, scoring='accuracy', n_iter=50, random_state=42
    )

    randomized_search.fit(X_train, y_train)

    results_df = pd.DataFrame(randomized_search.cv_results_)

    print(f"Best Hyperparameters: {randomized_search.best_params_}")

    y_train_pred = randomized_search.best_estimator_.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Train Accuracy: {train_accuracy}")

    y_test_pred = randomized_search.best_estimator_.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy}")
    
    return randomized_search.best_params_

# Assuming X_train, y_train, X_test, and y_test are available
params = tune_decision_tree_hyperparameters(X_train, y_train, X_test, y_test)


In [ ]:
print(params)

In [ ]:
from Utility_model_training import evaluate_classifier_with_stratified_kfold, evaluate_classifier_with_kfold_smote, evaluate_classifier_with_stratified_smote
from sklearn.tree import DecisionTreeClassifier
decision_tree_entropy = DecisionTreeClassifier(**params)

In [ ]:
evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, decision_tree_entropy, num_folds=10,save_path=path_smote_strat,model_name="dt_classifier_smote_stratified")
evaluate_classifier_with_stratified_kfold(X_train, y_train, X_test, y_test, decision_tree_entropy, num_folds=10,save_path=path_strat,model_name="dt_classifier_stratified")
evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, decision_tree_entropy, num_folds=10,save_path=path_smote,model_name="dt_classifier_smote")

In [ ]:
path_smote_strat_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote_strat/ROC_Curves"
path_strat_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/strat/ROC_Curves"
path_smote_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote/ROC_Curves"
from Utility_ROC_Curves import plot_roc_curves_multiclass_smote_strat,plot_roc_curves_multiclass_smote, plot_roc_curves_multiclass_strat
plot_roc_curves_multiclass_smote_strat(decision_tree_entropy, X, y, n_splits=10, save_folder=path_smote_strat_ROC, model_name="dt_KFold_SMOTE_strat")
plot_roc_curves_multiclass_strat(decision_tree_entropy, X, y, n_splits=10, save_folder=path_strat_ROC, model_name="dt_KFold_Strat")
plot_roc_curves_multiclass_smote(decision_tree_entropy, X, y, n_splits=10, save_folder=path_smote_ROC, model_name="dt_KFold_SMOTE")
